In [1]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl
print(pl.__version__)

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-ythvjh_6
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-ythvjh_6
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit 520c1e4713340f5bbf66de215471c2863e8fbdf2
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [2]:
import pandas as pd
import numpy as np
from tqdm. auto import tqdm
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn. functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from multiprocessing import cpu_count
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
#from pytorch_lightning.metrics.functional import accuracy
from sklearn.metrics import classification_report, confusion_matrix
import os
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils.rnn import pad_sequence
import warnings
warnings.filterwarnings("ignore")

from multiprocessing import cpu_count
import torchmetrics
from torchmetrics.functional import accuracy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
X_train = pd.read_csv("/kaggle/input/data-time-series/dataset_anuranjan.csv")

In [4]:
X_train.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,#,@,$,0,0
1,001519c8,2,4558,4962,404,Nonproduction,#,@,$,0,0
2,001519c8,3,106571,106571,0,Nonproduction,#,@,$,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
columns_to_encode = ['activity', 'up_event', 'down_event', 'text_change']

for column in columns_to_encode:
    X_train_encoded = pd.get_dummies(X_train[column], prefix=column)
    X_train = pd.concat([X_train, X_train_encoded], axis=1)
    X_train = X_train.drop(column, axis=1)

In [6]:
y_train = pd.read_csv('/kaggle/input/scores-time-series/train_scores.csv')

In [7]:
y_train.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [8]:
label_encoder=LabelEncoder ()
encoded_labels = label_encoder.fit_transform(y_train.score)

In [9]:
y_train["label"]= encoded_labels

In [10]:
y_train.drop('score', inplace=True, axis=1)

In [11]:
FEATURE_COLUMNS = X_train.columns.tolist()[2:]
FEATURE_COLUMNS

['down_time',
 'up_time',
 'action_time',
 'cursor_position',
 'word_count',
 'activity_Input',
 'activity_Nonproduction',
 'activity_Paste',
 'activity_Remove/Cut',
 'activity_Replace',
 'up_event_@',
 'up_event_Backspace',
 'up_event_Space',
 'up_event_q',
 'down_event_#',
 'down_event_Backspace',
 'down_event_Space',
 'down_event_q',
 'text_change_$',
 'text_change_q']

In [12]:
maxi = X_train.id.value_counts().max()
maxi

12876

In [13]:
sequences = []
for id, group in X_train.groupby("id"):
  sequence_features = group[FEATURE_COLUMNS]
  # Number of rows to add with zeros
  custom_rows = maxi - sequence_features.shape[0]

  # Creating a DataFrame with zeros
  zeros_df = pd.DataFrame(0, index=range(custom_rows), columns=sequence_features.columns)
  # Appending the DataFrame with zeros to the original DataFrame
  sequence_features = pd.concat([sequence_features, zeros_df], ignore_index=True)

  label= y_train[y_train.id == id].iloc[0].label
  sequences.append((sequence_features, label))

In [14]:
len(sequences)

2471

In [15]:
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2)

In [16]:
class SurfaceDataset(Dataset):
  def __init__ (self, sequences):
    self.sequences = sequences

  def __len__ (self):
    return len(self.sequences)

  def __getitem__ (self, idx):
    sequence, label = self.sequences [idx]
    return dict(
      sequence=torch.Tensor(sequence.to_numpy()),
      label=torch.tensor(label).long()
    )

In [17]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for batch in self.dl:
            # yield to_device(b, self.device)
            yield {key: value.to(self.device) if isinstance(value, torch.Tensor) else value for key, value in batch.items()}

    def __len__(self):
        """Number of batches"""
        return len(self.dl)



In [18]:
device = get_default_device()
device

device(type='cuda')

In [19]:
class SurfaceDataModule (pl.LightningDataModule):
  def __init__(self, train_sequences, test_sequences, batch_size):
    super().__init__()
    self.train_sequences = train_sequences
    self.test_sequences = test_sequences
    self.batch_size = batch_size

  def setup(self, stage=None):
    self.train_dataset = SurfaceDataset(self.train_sequences)
    self.test_dataset = SurfaceDataset(self. test_sequences)

  def train_dataloader (self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=cpu_count()
    )

  def val_dataloader (self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      shuffle=False,
      num_workers=cpu_count()
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      shuffle=False,
      num_workers=cpu_count()
    )

In [20]:
N_EPOCHS = 50
BATCH_SIZE = 8
data_module = SurfaceDataModule(train_sequences, test_sequences, BATCH_SIZE)

In [21]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_size, num_classes, d_model=8, nhead=2, num_layers=2):
        super(TimeSeriesTransformer, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.transformer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Change the sequence length dimension to be the first dimension
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Aggregate the sequence information
        x = self.fc(x)
        return x

In [22]:
class TimeSeriesClassifier(pl.LightningModule):
    def __init__(self, input_size, num_classes, d_model=8, nhead=2, num_layers=2, learning_rate=1e-3):
        super(TimeSeriesClassifier, self).__init__()
        self.model = TimeSeriesTransformer(input_size, num_classes, d_model, nhead, num_layers)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.num_classes=num_classes
        self.num_layers = num_layers

    def forward(self, x, labels = None):
        output = self.model(x)
        loss = 0
        if labels is not None:
          loss = self.criterion (output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        sequences, labels = batch['sequence'], batch['label']
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels, task='multiclass', num_classes=self.num_classes)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def validation_step(self, batch, batch_idx):
        sequences, labels = batch['sequence'], batch['label']
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels, task='multiclass', num_classes=self.num_classes)

        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def testing_step(self, batch, batch_idx):
        sequences, labels = batch['sequence'], batch['label']
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels, task='multiclass', num_classes=self.num_classes)

        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [23]:
model = TimeSeriesClassifier(
  input_size=len(FEATURE_COLUMNS),
  num_classes=len(label_encoder.classes_)
).to(device)

In [24]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [25]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="surface")

trainer = pl.Trainer(
  logger=logger,
  callbacks=checkpoint_callback,
  max_epochs=N_EPOCHS,
  devices=1,
  accelerator='gpu'

)

In [26]:
trainer.fit(model, data_module)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.88 GiB (GPU 0; 15.90 GiB total capacity; 9.92 GiB already allocated; 5.13 GiB free; 9.94 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF